In [1]:
# re-run this cell to load the rails environment

ENV["RAILS_ENV"] = 'development'
require ::File.expand_path('../config/environment',  __FILE__)

# pretty print JSON
# Usage: ppp Source.last
def ppp(obj, lang: 'json')
    guid = SecureRandom.hex
    html = "<pre><code class=\"#{lang}\" id=\"#{guid}\" style=\"font-size: 12px;\">#{CGI::escapeHTML(JSON.pretty_generate(obj.as_json))}</code></pre>"
    html += '<link rel="stylesheet" href="https://highlightjs.org/static/styles/railscasts.css">'
    html += '<script src="https://highlightjs.org/static/highlight.pack.js"></script>'
    html += "<script>hljs.highlightBlock(document.getElementById('#{guid}'));</script>"
    html += '<style type="text/css">.rendered_html code.hljs { padding: .5em; color: #e6e1dc; background-color: #232323; }</style>'

    IRuby.html html
end

:ppp

In [3]:
#
#
# see README for bigquery instructions
#
#

In [2]:
# import the leaguevids.csv file

require 'cgi'
url_regex = /\b(([\w-]+:\/\/?|www[.])[^\s()<>]+(?:\([\w\d]+\)|([^[:punct:]\s]|\/)))/
i = 0
j = 0
t = Time.now.to_i
skipped = {}
  
require 'csv'
  CSV.foreach('mmention.csv', :headers => true) do |csv_obj|
  # count total rows
  i += 1
  
  # filter subreddits
  csv_obj['subreddit'].downcase!
    next unless ['heroesofthestorm', 'nexusnewbies', 'competitivehots', 'leagueoflegends','loleventvods','summonerschool'].include?(csv_obj['subreddit'])
  
  # require fields
  skip = false
  ['id','body','link_id','created_utc','subreddit','author', 'score'].each do |required_key|
    if csv_obj[required_key].nil?
      skip = true
      skipped[required_key] ||= 0
      skipped[required_key] += 1
    end
  end
  next if skip
  
  # check if we've processed this comment
  comment = Comment.where('remote_id = ?', csv_obj['id']).first
  next unless comment.nil?
  
  # count rows we're processing
  j += 1
  
  # find youtube.com urls
  youtube_urls = csv_obj['body'].gsub(/&amp;/, '&').scan(url_regex).map {|r| r.first}.select{|url| url.include?("://www.youtube.com/")}

  # normalize
  youtube_urls = youtube_urls.map do |url|
    params = CGI::parse(url.split('?').last) rescue nil
    next if params.nil? || !params['v'] || !params['v'].first
    "https://www.youtube.com/watch?v=#{params['v'].first}"
  end
  
    # find youtu.be urls
  youtube_urls = csv_obj['body'].gsub(/&amp;/, '&').scan(url_regex).map {|r| r.first}.select{|url| url.include?("://youtu.be/")}

  # normalize
  youtube_urls = youtube_urls.map do |url|
    param = url.split('youtu.be/').last.split('?').first rescue nil
    next if param.nil?
    "https://www.youtube.com/watch?v=#{param}"
  end
  
  # unique
  youtube_urls.uniq!
  
  # skip this comment if we didn't find any urls
  next if youtube_urls.empty?
  
  # insert
  comment = Comment.create({
    :thread_url => "http://reddit.com/r/#{csv_obj['subreddit']}/comments/#{csv_obj['link_id'].split('_').last}",
    :remote_id => csv_obj['id'],
    :url => "http://reddit.com/r/#{csv_obj['subreddit']}/comments/#{csv_obj['link_id'].split('_').last}/a/#{csv_obj['id']}",
    :text => csv_obj['body'],
    :user => csv_obj['author'],
    :posted_at => Time.at(csv_obj['created_utc'].to_i).to_datetime,
    :subreddit => csv_obj['subreddit'],
    :score => csv_obj['score']
  })
  
  youtube_urls.each do |url|
    vid = Video.where('url = ?', url).first
    
    vid ||= Video.create({
      :url => url
    })
    
    reference = Reference.create({
      :video => vid,
      :comment => comment
    })
  end
end

t = Time.now.to_i - t

ppp ({ :total => i, :total_imported => j, :seconds => t })

"<pre><code class=\"json\" id=\"42d9215a07fc5ae04d9d091821e2ca09\" style=\"font-size: 12px;\">{\n  &quot;total&quot;: 4282070,\n  &quot;total_imported&quot;: 70679,\n  &quot;seconds&quot;: 1749\n}</code></pre><link rel=\"stylesheet\" href=\"https://highlightjs.org/static/styles/railscasts.css\"><script src=\"https://highlightjs.org/static/highlight.pack.js\"></script><script>hljs.highlightBlock(document.getElementById('42d9215a07fc5ae04d9d091821e2ca09'));</script><style type=\"text/css\">.rendered_html code.hljs { padding: .5em; color: #e6e1dc; background-color: #232323; }</style>"

In [5]:
# attributes missing that caused us to skip rows (hopefully empty)
ppp skipped

"<pre><code class=\"json\" id=\"3c27e975fd75fd4bf9d959eac73b268b\" style=\"font-size: 12px;\">{\n}</code></pre><link rel=\"stylesheet\" href=\"https://highlightjs.org/static/styles/railscasts.css\"><script src=\"https://highlightjs.org/static/highlight.pack.js\"></script><script>hljs.highlightBlock(document.getElementById('3c27e975fd75fd4bf9d959eac73b268b'));</script><style type=\"text/css\">.rendered_html code.hljs { padding: .5em; color: #e6e1dc; background-color: #232323; }</style>"

In [98]:
# get the most referenced videos
res = Reference.select('video_id, count(*) as count').group('video_id').order('count(*) DESC').limit(10).map do |ref|
  [ref.count, ref.video_id, Video.find(ref.video_id).url]
end

ppp res

"<pre><code class=\"json\" id=\"75fa8ba000e08fa0737f9899f4ac1a3c\" style=\"font-size: 12px;\">[\n  [\n    2704,\n    16,\n    &quot;https://www.youtube.com/watch?v=&quot;\n  ],\n  [\n    275,\n    52,\n    &quot;https://www.youtube.com/watch?v=dQw4w9WgXcQ&quot;\n  ],\n  [\n    67,\n    327,\n    &quot;https://www.youtube.com/watch?v=kB2UADNoRUA&quot;\n  ],\n  [\n    63,\n    647,\n    &quot;https://www.youtube.com/watch?v=x6WiBDTUWoA&quot;\n  ],\n  [\n    60,\n    1063,\n    &quot;https://www.youtube.com/watch?v=5UfO7qOHx5M&quot;\n  ],\n  [\n    56,\n    163,\n    &quot;https://www.youtube.com/watch?v=dgMMxTofZ5A&quot;\n  ],\n  [\n    56,\n    690,\n    &quot;https://www.youtube.com/watch?v=evjn7_tMRd4&quot;\n  ],\n  [\n    55,\n    1170,\n    &quot;https://www.youtube.com/watch?v=H1vl9MQEz_8&quot;\n  ],\n  [\n    54,\n    1159,\n    &quot;https://www.youtube.com/watch?v=ai6oMY4U0Yw&quot;\n  ],\n  [\n    53,\n    977,\n    &quot;https://www.youtube.com/watch?v=Lnowbp88lwI&quot;\n  ]\n]</code></pre><link rel=\"stylesheet\" href=\"https://highlightjs.org/static/styles/railscasts.css\"><script src=\"https://highlightjs.org/static/highlight.pack.js\"></script><script>hljs.highlightBlock(document.getElementById('75fa8ba000e08fa0737f9899f4ac1a3c'));</script><style type=\"text/css\">.rendered_html code.hljs { padding: .5em; color: #e6e1dc; background-color: #232323; }</style>"

In [102]:
# most referenced videos, limited to summoner school
res = Reference.joins("INNER JOIN comments ON (\"references\".comment_id = comments.id AND comments.subreddit = 'summonerschool')").select('video_id, count(*) as count').group('video_id').order('count(*) DESC').limit(10).map do |ref|
  [ref.count, ref.video_id, Video.find(ref.video_id).url]
end

ppp res

"<pre><code class=\"json\" id=\"6fdab614e1d94d745286e3a48fd3a0c9\" style=\"font-size: 12px;\">[\n  [\n    432,\n    16,\n    &quot;https://www.youtube.com/watch?v=&quot;\n  ],\n  [\n    31,\n    714,\n    &quot;https://www.youtube.com/watch?v=c3bmatXt6ds&quot;\n  ],\n  [\n    29,\n    163,\n    &quot;https://www.youtube.com/watch?v=dgMMxTofZ5A&quot;\n  ],\n  [\n    27,\n    12,\n    &quot;https://www.youtube.com/watch?v=6ywhB-2h2AE&quot;\n  ],\n  [\n    18,\n    4373,\n    &quot;https://www.youtube.com/watch?v=zkiCSUeOGTc&quot;\n  ],\n  [\n    18,\n    1064,\n    &quot;https://www.youtube.com/watch?v=1oPLc6DZvz8&quot;\n  ],\n  [\n    17,\n    552,\n    &quot;https://www.youtube.com/watch?v=ZQzB0HDstLY&quot;\n  ],\n  [\n    16,\n    1433,\n    &quot;https://www.youtube.com/watch?v=K4EfueGH7IY&quot;\n  ],\n  [\n    16,\n    417,\n    &quot;https://www.youtube.com/watch?v=cz7TWpRXyRY&quot;\n  ],\n  [\n    14,\n    995,\n    &quot;https://www.youtube.com/watch?v=hi63v_RnnBs&quot;\n  ]\n]</code></pre><link rel=\"stylesheet\" href=\"https://highlightjs.org/static/styles/railscasts.css\"><script src=\"https://highlightjs.org/static/highlight.pack.js\"></script><script>hljs.highlightBlock(document.getElementById('6fdab614e1d94d745286e3a48fd3a0c9'));</script><style type=\"text/css\">.rendered_html code.hljs { padding: .5em; color: #e6e1dc; background-color: #232323; }</style>"

In [106]:
comments = Reference.where('video_id = ?', 16).includes(:comment).map { |r| r.comment }
ppp comments.first

"<pre><code class=\"json\" id=\"5f477971324b77aa4476f446366e787c\" style=\"font-size: 12px;\">{\n  &quot;id&quot;: 13,\n  &quot;thread_url&quot;: &quot;http://reddit.com/leagueoflegends/comments/2zkkvd&quot;,\n  &quot;text&quot;: &quot;The official RiotGames channel is kinda good. [here](https://www.youtube.com/user/RiotGamesInc)\\n\\n\\n&quot;,\n  &quot;user&quot;: &quot;xRaiken&quot;,\n  &quot;posted_at&quot;: &quot;2015-03-19 10:31:22 UTC&quot;,\n  &quot;subreddit&quot;: &quot;leagueoflegends&quot;,\n  &quot;created_at&quot;: &quot;2015-08-02 07:40:45 UTC&quot;,\n  &quot;updated_at&quot;: &quot;2015-08-02 07:40:45 UTC&quot;,\n  &quot;thread_title&quot;: null,\n  &quot;url&quot;: &quot;http://reddit.com/leagueoflegends/comments/2zkkvd/a/e&quot;,\n  &quot;remote_id&quot;: &quot;cpjqy7e&quot;\n}</code></pre><link rel=\"stylesheet\" href=\"https://highlightjs.org/static/styles/railscasts.css\"><script src=\"https://highlightjs.org/static/highlight.pack.js\"></script><script>hljs.highlightBlock(document.getElementById('5f477971324b77aa4476f446366e787c'));</script><style type=\"text/css\">.rendered_html code.hljs { padding: .5em; color: #e6e1dc; background-color: #232323; }</style>"

"<pre><code class=\"json\" id=\"9fe78c94a7b17d62ddc5003444600388\" style=\"font-size: 12px;\">[\n  {\n    &quot;id&quot;: 13,\n    &quot;thread_url&quot;: &quot;http://reddit.com/leagueoflegends/comments/2zkkvd&quot;,\n    &quot;text&quot;: &quot;The official RiotGames channel is kinda good. [here](https://www.youtube.com/user/RiotGamesInc)\\n\\n\\n&quot;,\n    &quot;user&quot;: &quot;xRaiken&quot;,\n    &quot;posted_at&quot;: &quot;2015-03-19 10:31:22 UTC&quot;,\n    &quot;subreddit&quot;: &quot;leagueoflegends&quot;,\n    &quot;created_at&quot;: &quot;2015-08-02 07:40:45 UTC&quot;,\n    &quot;updated_at&quot;: &quot;2015-08-02 07:40:45 UTC&quot;,\n    &quot;thread_title&quot;: null,\n    &quot;url&quot;: &quot;http://reddit.com/leagueoflegends/comments/2zkkvd/a/e&quot;,\n    &quot;remote_id&quot;: &quot;cpjqy7e&quot;\n  },\n  {\n    &quot;id&quot;: 16,\n    &quot;thread_url&quot;: &quot;http://reddit.com/leagueoflegends/comments/3ah87u&quot;,\n    &quot;text&quot;: &quot;https://www.youtube.com/channel/UC8Ap_5x2SSiowTjkjeGssWQ&quot;,\n    &quot;user&quot;: &quot;worstadceunegg&quot;,\n    &quot;posted_at&quot;: &quot;2015-06-20 03:57:03 UTC&quot;,\n    &quot;subreddit&quot;: &quot;leagueoflegends&quot;,\n    &quot;created_at&quot;: &quot;2015-08-02 07:40:45 UTC&quot;,\n    &quot;updated_at&quot;: &quot;2015-08-02 07:40:45 UTC&quot;,\n    &quot;thread_title&quot;: null,\n    &quot;url&quot;: &quot;http://reddit.com/leagueoflegends/comments/3ah87u/a/6&quot;,\n    &quot;remote_id&quot;: &quot;csclyl6&quot;\n  },\n  {\n    &quot;id&quot;: 18,\n    &quot;thread_url&quot;: &quot;http://reddit.com/leagueoflegends/comments/2vvx6l&quot;,\n    &quot;text&quot;: &quot;It&#39;s from their Team Previews and this is supposed to be the CLG video I guess. Rest of the teams are here (https://www.youtube.com/playlist?list=PLPZ7h6L6LC7UZZm8srKZHyftnLHwmp4tL)&quot;,\n    &quot;user&quot;: &quot;imonfireahh&quot;,\n    &quot;posted_at&quot;: &quot;2015-02-14 18:09:19 UTC&quot;,\n    &quot;subreddit&quot;: &quot;leagueoflegends&quot;,\n    &quot;created_at&quot;: &quot;2015-08-02 07:40:46 UTC&quot;,\n    &quot;updated_at&quot;: &quot;2015-08-02 07:40:46 UTC&quot;,\n    &quot;thread_title&quot;: null,\n    &quot;url&quot;: &quot;http://reddit.com/leagueoflegends/comments/2vvx6l/a/o&quot;,\n    &quot;remote_id&quot;: &quot;coledpo&quot;\n  },\n  {\n    &quot;id&quot;: 21,\n    &quot;thread_url&quot;: &quot;http://reddit.com/summonerschool/comments/34exzs&quot;,\n    &quot;text&quot;: &quot;I found [this channel](https://www.youtube.com/user/unswlolsoc/videos) awhile ago. It goes into great detail on positioning and trading. It also covers a wealth of other things. &quot;,\n    &quot;user&quot;: &quot;Toxicfunk314&quot;,\n    &quot;posted_at&quot;: &quot;2015-04-30 23:23:59 UTC&quot;,\n    &quot;subreddit&quot;: &quot;summonerschool&quot;,\n    &quot;created_at&quot;: &quot;2015-08-02 07:40:46 UTC&quot;,\n    &quot;updated_at&quot;: &quot;2015-08-02 07:40:46 UTC&quot;,\n    &quot;thread_title&quot;: null,\n    &quot;url&quot;: &quot;http://reddit.com/summonerschool/comments/34exzs/a/b&quot;,\n    &quot;remote_id&quot;: &quot;cquf10b&quot;\n  },\n  {\n    &quot;id&quot;: 26,\n    &quot;thread_url&quot;: &quot;http://reddit.com/leagueoflegends/comments/32ntz0&quot;,\n    &quot;text&quot;: &quot;Take this as advice and suggestions and not as bashing. The best place to learn anything about league, imho, is the long and great videos from [Unswlolsoc&#39;s youtube channel](https://www.youtube.com/user/unswlolsoc). It is quite dated but the information is 99% timeless. If you guys had something of this sort I&#39;d probably sub for at least a month.&quot;,\n    &quot;user&quot;: &quot;FuujinSama&quot;,\n    &quot;posted_at&quot;: &quot;2015-04-15 22:30:47 UTC&quot;,\n    &quot;subreddit&quot;: &quot;leagueoflegends&quot;,\n    &quot;created_at&quot;: &quot;2015-08-02 07:40:46 UTC&quot;,\n    &quot;updated_at&quot;: &quot;2015-08-02 07:40:46 UTC&quot;,\n   